<div style="text-align: center;">
    <h1 style="color: #FF6347;">LangChain Agents</h1>
</div>

<div style="text-align: center;">
    <img src="https://i.giphy.com/media/v1.Y2lkPTc5MGI3NjExODI5b2w0OHc3eWViM29maG1oZDluMGI2enJoNmpkbDV4eWxqM3F1aSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/yTZNtbeccaZLgnxEVC/giphy.gif" alt="NLP Gif" style="width: 300px; height: 150px; object-fit: cover; object-position: center;">
</div>

By combining tools like web search, vector-based retrieval, and generative models, LangChain agents excel in creating versatile, real-world applications for information retrieval, question answering, and decision-making systems.

<h3 style="color: #FF8C00;">By the End of This Lesson, You'll:</h3>

- Understand the fundamentals of LangChain agents and their applications.
- Learn how to integrate multiple tools into a single AI system.
- Use embeddings to create efficient vector representations for documents.
- Apply vector-based similarity search to retrieve relevant data.
- Generate responses dynamically using generative models like OpenAI's GPT.
- Maintain conversational memory for multi-turn, context-aware interactions.

---

<h2 style="color: #FF6347;">LangChain Agent Overview</h2>

LangChain agents are designed to dynamically decide which tools to use based on user inputs. They enable seamless integration of document retrieval, external APIs, and generative models, creating a highly flexible conversational system.

<h3 style="color: #FF8C00;">Key Features of LangChain Agents</h3>

- **Multi-Tool Integration**:
  - Combine tools such as search engines, vector stores, and APIs for diverse capabilities.
  - Example: Retrieve information from web documents and query pre-trained embeddings simultaneously.

- **Document Retrieval**:
  - Utilize vector databases (e.g., FAISS, ChromaDB) to find relevant document chunks based on user queries.
  - Example: Find specific sections of a technical document to answer a query.

- **Generative Models**:
  - Use models like OpenAI's GPT to generate human-like, context-aware responses.
  - Example: Combine retrieved documents into a prompt for GPT to create accurate and detailed answers.

- **Conversational Memory**:
  - Maintain context across multiple turns in a conversation for a more natural and engaging user experience.
  - Example: Remember the user's name and preferences during an ongoing chat.

<h2 style="color: #FF6347;">Setting Up the GPT Model</h2>

In [1]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Define the model and API key
model_gpt = "gpt-4o"
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
client = ChatOpenAI(
    model=model_gpt, 
    temperature=0.7, # Increase creativity
    max_tokens=4000, # Allow for longer responses
    frequency_penalty=0.5, # Reduce repetition
    presence_penalty=0.6, # Encourage new topics 
    api_key=api_key
)
print("GPT Model Initialized")

GPT Model Initialized


<h2 style="color: #FF6347;">Loading a Prompt from LangChain Hub</h2>

In [4]:
from langchain import hub

In [5]:
# Pull a predefined prompt
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages = "Hey, I'm a language model trained by OpenAI. I can help you with a variety of tasks. What would you like to know?"
prompt.messages

/opt/anaconda3/envs/genai/lib/python3.10/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"Hey, I'm a language model trained by OpenAI. I can help you with a variety of tasks. What would you like to know?"

<h2 style="color: #FF6347;">Loading and Splitting Documents</h2>

In [6]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


<h3 style="color: #FF8C00;">Loading the Documents</h3>

In [7]:
loader = WebBaseLoader("https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html")
docs = loader.load()

<h3 style="color: #FF8C00;">Documents into chunks</h3>

In [8]:
# Split the document into chunks
documents = RecursiveCharacterTextSplitter(
    chunk_size=10000, 
    chunk_overlap=0
).split_documents(docs)

print(f"Loaded and split {len(documents)} document chunks.")

Loaded and split 3 document chunks.


<h2 style="color: #FF6347;">Creating a Vector Store</h2>

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [10]:
# pip install faiss-cpu

<h2 style="color: #FF8C00;">FAISS</h2>

FAISS (Facebook AI Similarity Search) is a library developed by Meta AI for efficient similarity search and clustering of dense vector embeddings. It is designed to perform approximate and exact nearest neighbor searches, enabling rapid and scalable vector retrieval for various applications.

- **Benefits:**
  - Ideal for in-memory workloads where speed is critical.
  - Scales well with large datasets, ensuring rapid similarity search.
  - Integrates seamlessly with embedding models like OpenAI and SentenceTransformers.

- **Applications:**
  - Document retrieval for Retrieval-Augmented Generation (RAG).
  - Product or content recommendation systems.
  - Semantic search and clustering of textual data.

- **Comparison with ChromaDB**:
  - **FAISS**: Focuses on in-memory operations for high-speed searches.
  - **ChromaDB**: Designed for persistent storage and metadata-based filtering, better suited for long-term workflows.

In [11]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings(api_key=api_key))
retriever = vector.as_retriever()
retriever
print("Vector store created and retriever initialized.")

Vector store created and retriever initialized.


<div style="text-align: center;">
    <img src="https://yt3.googleusercontent.com/ytc/AIdro_nojGiB8pRf13nWYqYP4WD_orQXpQKyqnP7KCoMWkpGRJc=s900-c-k-c0x00ffffff-no-rj" alt="NLP Gif" style="width: 180px">
</div>

<h2 style="color: #FF6347;">Adding Tools: DuckDuckGoSearchRun</h2>

The **DuckDuckGoSearchRun** is a LangChain tool that allows you to perform real-time searches using the DuckDuckGo search engine. It provides up-to-date information by querying the web directly, making it a valuable tool for handling questions about current events or dynamic topics not covered in static datasets.

- **What It Does**:
  - Sends queries to the DuckDuckGo search engine.
  - Retrieves and processes search results in real-time.
  - Returns concise, relevant information to the user.

- **When to Use It**:
  - For questions requiring the most recent information (e.g., news, trends, or updates).
  - When static embeddings or pre-trained models lack sufficient context or coverage.

- **Key Benefits**:
  - Maintains user privacy, as DuckDuckGo does not track user queries.
  - Simple integration into LangChain workflows for real-time web searches.


In [12]:
# pip install -U duckduckgo-search

In [13]:
from langchain_community.tools import DuckDuckGoSearchRun

In [14]:
# Initialize DuckDuckGo search tool
search = DuckDuckGoSearchRun()

<div style="text-align: center;">
    <img src="https://leerburg.com/Photos/articles/main/hold-exercise.jpg" alt="NLP Gif" style="width: 300px">
</div>

<h2 style="color: #FF6347;">Retriever Tool</h2>

The **Retriever Tool** is a LangChain utility for searching and fetching information from a pre-initialized retriever. It acts as a bridge between the query and the stored vector database or search index.

- **What It Does**:
  - Facilitates querying the retriever (e.g., FAISS or ChromaDB) for specific information.
  - Retrieves relevant documents or data chunks based on the input query.
  - Returns results optimized for further processing by downstream tools like LLMs.


In [15]:
from langchain.tools.retriever import create_retriever_tool

In [16]:
# Create a retriever tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about DataFrame. Use this tool for any DataFrame-related questions.",
)

In [17]:
# Combine tools
tools = [search, retriever_tool]
print(f"Tools initialized: {len(tools)}")

Tools initialized: 2


<div style="text-align: center;">
    <img src="https://ainiro.io/assets/images/blog/ai-agents-the-future-of-work.webp" alt="NLP Gif" style="width: 350px">
</div>

<h2 style="color: #FF6347;">Creating a Tool-Calling Agent</h2>

The **Tool-Calling Agent** is a core component in LangChain that combines tools, prompts, and an LLM to process user queries. It intelligently selects and utilizes the appropriate tools to provide accurate and context-aware responses.


In [18]:
from langchain.agents import create_tool_calling_agent

In [51]:
# Create an agent that uses the tools
agent = create_tool_calling_agent(client, tools, prompt)
print("Agent created.")

Agent created.


In [20]:
# parameters are:
# agent = create_tool_calling_agent(client, tools, prompt, max_turns=5, max_turn_length=1000, max_response_length=1000, max_response_turns=1, max_response_time=10)
# max_turns: maximum number of turns in a conversation
# max_turn_length: maximum number of characters in a turn
# max_response_length: maximum number of characters in a response
# max_response_turns: maximum number of turns in a response
# max_response_time: maximum time in seconds for a response

<h2 style="color: #FF6347;">Using the Agent Executor</h2>

The **Agent Executor** is used to invoke queries and retrieve responses from the agent. It processes input queries, utilizes the appropriate tools, and generates structured outputs. This code demonstrates how to handle multiple queries with error handling for robustness.


In [52]:
from langchain.agents import AgentExecutor

In [53]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [54]:
print(prompt.messages)

Hey, I'm a language model trained by OpenAI. I can help you with a variety of tasks. What would you like to know?


In [63]:
import asyncio

async def main():
    try:
        # Example input format with a list of messages
        input_messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "How much is 2 + 2?"}
        ]
        
        result = await agent_executor.ainvoke({"input": input_messages})
        print(result)
    except Exception as e:
        print(f"An error occurred: {e}")
        # Print more details about the error
        import traceback
        traceback.print_exc()

# Run the async main function
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# Example
agent_executor.invoke({"input": "What is a DataFrame?"})
agent_executor.invoke({"input": "How to use a Series in pandas?"})

<h2 style="color: #FF6347;">Adding Conversational Memory</h2>

In [38]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [39]:
# Create a simple in-memory chat history
message_history = ChatMessageHistory()

# Add memory to the agent
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# Example query with memory
agent_with_chat_history.invoke(
    {"input": "Hi, I am Michael and I am transitioning from cooking to tech."},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...


ValueError: Unexpected input: H

<h2 style="color: #FF6347;">Testing Conversational Memory</h2>

In [ ]:
# Follow-up queries with memory
agent_with_chat_history.invoke(
    {"input": "What’s my name?"},
    config={"configurable": {"session_id": "<foo>"}},
)

agent_with_chat_history.invoke(
    {"input": "How can Ironhack help me in my career?"},
    config={"configurable": {"session_id": "<foo>"}},
)

<h2 style="color: #FF6347;">Exploring Use Cases</h2>

In [ ]:
# Use the agent for different scenarios
agent_executor.invoke({"input": "How can I create a pie chart in PowerBI?"})
agent_with_chat_history.invoke(
    {"input": "Can you tell me what LangChain does?"},
    config={"configurable": {"session_id": "<your-session-id>"}}
)